# 2017-05-15 : running the main openRetina scripts
---

Progress of my work : After some weeks of reverse-engineering the OpenRetina scripts, I understood them enough to work direclty on them. L. Perrinet and I built some testing scripts that allowed me to try these scripts. The first one, with a fully-noised input, was run successfully. I still have some problems to run the second one, that use a local camera as an input.

- [ ] Run the local testing script
    - [ ] Solve the empty capture problem
    
Since the code() function seems to receive empty inputs, the problem may come from the run() function.  
Trying to find the source of that problem.  

    def run(self):
        (...)
        print('dbg_key1', self.frame)
        data = self.code(self.frame)
        (...)
        
Output : 

    dbg_key1 None
    
Then :

    def grab(self):
        (...)
        print('dbg_key1', frame)
        print('dbg_key2', self.cap.read())
        (...)
        
Output : 
    
    dbg_key1 None
    dbg_key2 (False, None)
    
The problem seem to come from the PhotoReceptor() class, as trying to call it alone don't work : 

    from openRetina import openRetina
    from openRetina import PhotoReceptor
    import cv2

    cam = PhotoReceptor(640,480)
    cam.grab()
    
Output : 

    Unable to capture video
    dbg_key1 (False, None)

I had some internet problems that slowed down my work. My computer get randomly disconnected, forcing me to restart it to make it work again.  
The script problem come from the PhotoReceptor class, as : 

    (...)
    if not self.cap.isOpened(): 
        print('Camera is not opened')
        stop
    print('dbgkey1')
    (...)
    
Output :

    Camera is not opened
    
I'm missing something. While my own *camera.py* and the official *openRetina* PhotoReceptor classes are fairly similar, the openRetina one don't work. 

    Before a downscale of 1 dim1 : 480, dim2 : 640
    Using OpenCV
    After a downscale of 1, dim1 : 0.0, dim2 : 0.0
    
The null dimensions surely come from the closed camera. Why is the camera not opening?  

- [ ] Run the local testing script
    - [ ] Solve the non-opening camera problem

OpenCV documentation suggested to use the following function to force it, but it doesn't change anything (see : http://opencv-python-tutroals.readthedocs.io/en/latest/py_tutorials/py_gui/py_video_display/py_video_display.html#capture-video-from-camera) :

    self.cap.open(0)
    
When used through Cheese (capturing program), the camera open and record just fine.  
This command line give informations about the devices connected to the computer's USB bus, including the camera (first one to appear, see : https://help.ubuntu.com/community/Webcam) :

    lsusb -v
    
Even when using a simple script that I already used and that was working, the camera don't launch and no error is given :

    import cv2

    cap = cv2.VideoCapture(0)
    if not cap.isOpened():
        print('Cam not opnd')
    while(True):
        ret,frame = cap.read()
        cv2.imshow('frame', frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

I'm using the last openCV version (3.2.0.7), maybe that's what is causing that bug. But the VideoCapture is still called the same way than the previous versions I used (API of the function for that version : http://docs.opencv.org/3.0-beta/modules/videoio/doc/reading_and_writing_video.html?highlight=videocapture#videocapture-videocapture).  
    The fact that the camera isn't opening on my personnal computer that use the same openCV version reinforce the idea that it is linked to the openCV version used (as suggested here : http://answers.opencv.org/question/88857/videocapture-is-not-working-in-opencv-300-or-the-310-in-windows-7/ and suggested by lot of recent questions on internet about the same issue, eg.: http://answers.opencv.org/question/136678/opencv-error-assertion-failed-sizewidth0-sizeheight0-in-cvimshow-python/, https://github.com/opencv/opencv/issues/8471 and https://github.com/opencv/opencv/issues/8358).  
I need to ask L. Perrinet which version is installed on his computer, since the scripts worked on it.  
All these links suggest that the issue is caused by the version 3.2.x, so I tried to downgrade to 3.1.x, the previous version :

    pip3 uninstall opencv-python
    pip install opencv-python==3.1.0 --user
    
But it didn't solved the issue.  
I'm working on that issue since this morning and I'm still not able to solve it.  I'll talk with L. Perrinet about that.

In the meantime, I tried to run *2017-05-15_using_openretina* and *2017-05-15_using_openretina_server*.

- [X] Run the openRetina script

It works! A window is opening and show several frames captured, with a red/blue color coding on a black background, as programmed by the openRetina script.  
Still, several non-critical errors are showing, that may decrease the script performance.

    *** Pi side ***
    
    mmal: mmal_vc_port_enable: failed to enable port vc.null_sink:in:0(OPQV): ENOSPC
    mmal: mmal_port_enable: failed to enable connected port (vc.null_sink:in:0(OPQV))0x16dcd70 (ENOSPC)
    mmal: mmal_connection_enable: output port couldn't be enabled
    
    💀  Could not load visualisation
    
    Traceback (most recent call last):
      File "2017-05-15_using_openretina.py", line 13, in <module>
        output=['stream'] # output: can be stream, display, ...
      File "/home/pi/openRetina/src/openRetina.py", line 146, in __init__
        self.camera = PhotoReceptor(self.w, self.h)
      File "/home/pi/openRetina/src/openRetina.py", line 46, in __init__
        self.cap = picamera.PiCamera(cam_id)
      File "/usr/lib/python3/dist-packages/picamera/camera.py", line 433, in __init__
        self._init_preview()
      File "/usr/lib/python3/dist-packages/picamera/camera.py", line 513, in _init_preview
        self, self._camera.outputs[self.CAMERA_PREVIEW_PORT])
      File "/usr/lib/python3/dist-packages/picamera/renderers.py", line 558, in __init__
        self.renderer.inputs[0].connect(source).enable()
      File "/usr/lib/python3/dist-packages/picamera/mmalobj.py", line 2212, in enable
        prefix="Failed to enable connection")
      File "/usr/lib/python3/dist-packages/picamera/exc.py", line 184, in mmal_check
        raise PiCameraMMALError(status, prefix)
    picamera.exc.PiCameraMMALError: Failed to enable connection: Out of resources


# 2017-05-16
---

These errors seem to be linked to picamera's mmalobj module (complete API : http://picamera.readthedocs.io/en/latest/api_mmalobj.html). They don't show at every run.  

Now that the openRetina script is correctly running, I can begin to apply modifications to it. In accordance with L. Perrinet, I'll insert a simple script to capture and save a video file and then modify the script to allow it to work with that file, instead of a stream. That'll allow to bypass the networking limitations, like bandwidth consumption.  

I'll then capture a video with a stable background and a moving object (eg. a hand), so some of the pixels will change value while other will keep the same over time. The objective is to modify the program to only code (send to server) information about the changing pixels instead of coding the luminance contrast as it does now.

About the poster that'll act as my final presentation, L. Perrinet thinks that it may be better to show what I've learned (python classes, git and ssh use, raspberry manipulation, biomimetism, ...) instead of my results.

- [ ] Update openRetina script to get more informative prints

- [ ] Write a simple script to capture a video to a file

- [ ] Capture a video with a stable background and moving object

# Outline
---

- [ ] Run the local testing script
    - [ ] Solve the empy capture problem
    - [ ] Solve the non-opening camera problem
- [X] Run the openRetina script
- [ ] Update openRetina script to get more informative prints
- [ ] Write a simple script to capture a video to a file
- [ ] Capture a video with a stable background and moving object